In [1]:
%run Hooklog3.ipynb

In [2]:
import os

In [3]:
class NestedHooklog3(Hooklog3):
    
    def __init__(self, path, par):
        self.path = path
        self.par = par
        self.li = list()
        self.length = 0
        
        self._parseDigitname()
        self.__parseHooklog() # use NestedHooklog3's __parseHooklog, not Hooklog3
        self.__deleteNested()
    
    def __str__(self):
        return "class NestedHooklog3, %s, par = %d, len = %d, digit name = %s" % (self.path, self.par, self.length, self.digitname)
    
    # private functions
    def __parseHooklog(self):
        if self.isExist():
            handle = open(self.path, 'rb')
            index = 0 # MIKE: 20170616, Nested return index
            
            while 1:
                (tick, api) = self._getNextPair(handle)
                if tick == 0:
                    break # end of file or fail to read
                self.li.append((tick, api, index))
                self.length += 1
                index += 1
            if handle: handle.close()
            
            # MIKE: 20170616, will sorted by nested structure by ____deleteNested, not tick anymore!
            self.li.sort(key = lambda tup: tup[0]) # sort by tick
            
        else:
            pass
    
    def __deleteNested(self):
        # MIKE: uncomment for debug nestedhooklog
        #for i, (t, a, r) in enumerate(self.li):
        #    print(i, t, a, r)
        
        pre_c, pre_r = -1, -1
        deleted = 0
        length = len(self.li)
        for call in range(length): # MIKE: dont use len() here, list length changes
            index = call - deleted
            (tick, api, ret) = self.li[index]
            
            if pre_c == pre_r or call == ret: # ! not nested
                # for compatibility remove ret
                self.li[index] = (tick, api)
            else:
                del self.li[index]
                deleted += 1
            pre_c, pre_r = call, ret
            
        # update length
        self.length = len(self.li)